In [1]:
%classpath add mvn org.uma.jmetal jmetal-problem 5.5.2
%classpath add mvn org.uma.jmetal jmetal-algorithm 5.5.2
%classpath add mvn org.uma.jmetal jmetal-core 5.5.2
%classpath add mvn org.uma.jmetal jmetal-exec 5.5.2

## Plotting Stuff

In [5]:
import org.uma.jmetal.util.front.imp.ArrayFront
import org.uma.jmetal.solution.PermutationSolution;

def iterator(size,valuefun){
    r = new ArrayList()
    for(int i = 0;i<size;i++) {
        r.add(valuefun.call(i))
    }
    return r
}

plotFronts = {obj,refFile=null ->
    plot = new Plot()    

    if(refFile != null) { 
        ref = new ArrayFront(refFile)
        xRef = iterator(ref.getNumberOfPoints(),{i -> ref.getPoint(i).getValues()[0]})
        yRef = iterator(ref.getNumberOfPoints(),{i -> ref.getPoint(i).getValues()[1]})    
        plot  << new Points(x: xRef, y: yRef, displayName: "Reference Front")
    }
    
    // Sorted to make lines that make sense
    obj.sort(new Comparator<PermutationSolution<Integer>>() {
        public int compare(PermutationSolution<Integer> sol1, PermutationSolution<Integer> sol2) {
            return sol2.getObjective(0).compareTo(sol1.getObjective(0));
        }
    });
    
    xObj = iterator(obj.size(),{i -> obj.get(i).getObjective(0)})
    yObj = iterator(obj.size(),{i -> obj.get(i).getObjective(1)})
    plot << new Points(x: xObj, y: yObj, displayName: "Objective Function (Points)")
    plot << new Line(x: xObj, y: yObj, displayName: "Objective Function (Line)")
    plot.display()
}

script1529333231952$_run_closure1@59b38818

## Problem Definiton

In [156]:
import org.uma.jmetal.problem.impl.AbstractIntegerPermutationProblem;
import org.uma.jmetal.solution.PermutationSolution;
import org.uma.jmetal.util.JMetalException;

import java.io.BufferedReader;
import java.io.IOException;
import java.io.InputStream;
import java.io.InputStreamReader;
import java.io.StreamTokenizer;

public class MultiobjectiveTSPRO extends AbstractIntegerPermutationProblem {
  protected int         numberOfCities ;
  protected double [][] distanceMatrix ;
  protected double [][] costMatrix;

  /**
   * Creates a new MultiobjectiveTSP problem instance
   */

  public MultiobjectiveTSPRO()  {
    System.out.print("Go fuck yourself!")
  }
    
   public MultiobjectiveTSPRO(String distanceFile, String costFile) throws IOException {
    distanceMatrix = readProblem(distanceFile) ;
    costMatrix     = readProblem(costFile);

    setNumberOfVariables(numberOfCities);
    setNumberOfObjectives(2);
    setName("MultiobjectiveTSP");
  }

  /** Evaluate() method */
  public void evaluate(PermutationSolution<Integer> solution){
    double fitness1   ;
    double fitness2   ;

    fitness1 = 0.0 ;
    fitness2 = 0.0 ;

    for (int i = 0; i < (numberOfCities - 1); i++) {
      int x ;
      int y ;

      x = solution.getVariableValue(i) ;
      y = solution.getVariableValue(i+1) ;

      fitness1 += distanceMatrix[x][y] ;
      fitness2 += costMatrix[x][y];
    }
    int firstCity ;
    int lastCity  ;

    firstCity = solution.getVariableValue(0) ;
    lastCity = solution.getVariableValue(numberOfCities - 1) ;

    fitness1 += distanceMatrix[firstCity][lastCity] ;
    fitness2 += costMatrix[firstCity][lastCity];

    solution.setObjective(0, fitness1);
    solution.setObjective(1, fitness2);
  }

  private double [][] readProblem(String fileName) {
    double [][] matrix = null;
    def c = [];
      
    File file = new File(fileName)
    boolean valuesNextline = false
    
    file.eachLine { line ->
        
        if(valuesNextline && !line.contains("EOF")) {
            String[] vals = line.split()
            c << vals[1].toDouble()
            c << vals[2].toDouble()
        } else if(line.contains("DIMENSION")) {
            numberOfCities = line.split()[1].toInteger()
            matrix = new double[numberOfCities][numberOfCities]
        } else if(line.contains("SECTION"))
            valuesNextline = true

    }
      double dist ;
      for (int k = 0; k < numberOfCities; k++) {
        matrix[k][k] = 0;
        for (int j = k + 1; j < numberOfCities; j++) {
          dist = Math.sqrt(Math.pow((c[k*2]-c[j*2]),2.0) +
              Math.pow((c[k*2+1]-c[j*2+1]), 2));
          dist = (int)(dist + 0.5);
          matrix[k][j] = dist;
          matrix[j][k] = dist;
        }
      }
    return matrix;
  }

  public int getPermutationLength() {
    return numberOfCities ;
  }
    
  public double[][] getDistanceMatrix() {
    return distanceMatrix;
  }

  public double[][] getCostMatrix() {
    return costMatrix;
  }
}

Go fuck yourself!

null

## NSGAII + Initialization

In [159]:
import org.uma.jmetal.algorithm.multiobjective.nsgaii.NSGAIIBuilder;
import org.uma.jmetal.operator.CrossoverOperator;
import org.uma.jmetal.operator.MutationOperator;
import org.uma.jmetal.operator.SelectionOperator;

import org.uma.jmetal.operator.impl.crossover.PMXCrossover;
import org.uma.jmetal.operator.impl.mutation.PermutationSwapMutation;
import org.uma.jmetal.operator.impl.selection.BinaryTournamentSelection;

import org.uma.jmetal.problem.PermutationProblem;
import org.uma.jmetal.problem.multiobjective.MultiobjectiveTSP;
import org.uma.jmetal.solution.PermutationSolution;

import org.uma.jmetal.util.AbstractAlgorithmRunner;
import org.uma.jmetal.util.AlgorithmRunner;
import org.uma.jmetal.util.fileoutput.SolutionListOutput;
import org.uma.jmetal.util.comparator.RankingAndCrowdingDistanceComparator;
import org.uma.jmetal.util.fileoutput.impl.DefaultFileOutputContext;

import org.uma.jmetal.util.pseudorandom.JMetalRandom;

JMetalRandom.getInstance().setSeed(100L);


//problem = new MultiobjectiveTSP("/tspInstances/kroA100.tsp", "/tspInstances/kroB100.tsp");

problem = new MultiobjectiveTSPRO("toyA.tsp","toyB.tsp");

crossover = new PMXCrossover(0.9) ;

double mutationProbability = 0.2 ;
mutation = new PermutationSwapMutation<Integer>(mutationProbability) ;

selection = new BinaryTournamentSelection<PermutationSolution<Integer>>(new RankingAndCrowdingDistanceComparator<PermutationSolution<Integer>>());

algorithm = new NSGAIIBuilder<PermutationSolution<Integer>>(problem, crossover, mutation)
        .setSelectionOperator(selection)
        .setMaxEvaluations(10000)
        .setPopulationSize(100)
        .build() ;

algorithmRunner = new AlgorithmRunner.Executor(algorithm)
        .execute() ;

population = algorithm.getResult();
computingTime = algorithmRunner.getComputingTime() ;

new SolutionListOutput(population)
        .setSeparator("\t")
        .setVarFileOutputContext(new DefaultFileOutputContext("VAR.tsv"))
        .setFunFileOutputContext(new DefaultFileOutputContext("FUN.tsv"))
        .print();

plotFronts(population)

TableDisplay(problem.getCostMatrix())

"""
Total execution time: ${computingTime} ms
Random seed: ${JMetalRandom.getInstance().getSeed()}
Objectives values have been written to file FUN.tsv
Variables values have been written to file VAR.tsv
"""

groovy.lang.GroovyRuntimeException:  Could not find matching constructor for

In [64]:


this.getClass().getResource(file).withInputStream { ins ->
    InputStreamReader isr = new InputStreamReader(ins);
    BufferedReader br = new BufferedReader(isr);
    StreamTokenizer token = new StreamTokenizer(br);
}
token

java.lang.NullPointerException:  Cannot invoke method withInputStream() on null object

In [72]:
this.getClass().getResource("toyA.tsp")

new File("toyA.tsp").eachLine {  
    line -> 
    if(line == line){
        println "line : $line";     
    }
}

line : NAME: toyA
line : TYPE: TSP
line : COMMENT: 10-city problem A (Krolak/Felts/Nelson)
line : DIMENSION: 10
line : EDGE_WEIGHT_TYPE : EUC_2D
line : NODE_COORD_SECTION
line : 1 1 1
line : 2 1 2
line : 3 1 3
line : 4 1 4
line : 5 1 5
line : 6 3 1
line : 7 3 2
line : 8 3 3
line : 9 3 4
line : 10 3 5
line : EOF


null